In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing,

# list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

 # Load the given datasets

In [ ]:
train = pd.read_csv('/kaggle/input/data-storm-10/credit_card_default_train.csv')
test = pd.read_csv('/kaggle/input/data-storm-10/credit_card_default_test.csv')

In [ ]:
testc=test.copy()

In [ ]:
train.head(100)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()#check for null values

# Preprocessing

In [ ]:
# for encoding the data given
cleanup = {"Gender" : {"M" : 0,
                       "F":1
                      },
           "EDUCATION_STATUS" : {"High School" : 0,
                                 "Graduate" : 1,
                                 "Other" : 2
                                },
           "MARITAL_STATUS" : {"Single" : 0,
                               "Other" : 1
                              },
           "AGE" : {"Less than 30" : 0,
                    "31-45" : 1,
                    "46-65" : 2,
                    "More than 65" : 3
                   },
           "PAY_JULY" : {8 : 9,
                         7 : 9,
                         6 : 9,
                         5 : 9,
                         4 : 9,
                         3 : 9,
                         2 : 9,
                         1 : 9,
                         -1 : 0,
                         -2 : 0,
                        },
           "PAY_AUG" : {8 : 9,
                         7 : 9,
                         6 : 9,
                         5 : 9,
                         4 : 9,
                         3 : 9,
                         2 : 9,
                         1 : 9,
                         -1 : 0,
                         -2 : 0,
                        },
           "PAY_SEP" : {8 : 9,
                         7 : 9,
                         6 : 9,
                         5 : 9,
                         4 : 9,
                         3 : 9,
                         2 : 9,
                         1 : 9,
                         -1 : 0,
                         -2 : 0,
                        },
           "PAY_OCT" : {8 : 9,
                         7 : 9,
                         6 : 9,
                         5 : 9,
                         4 : 9,
                         3 : 9,
                         2 : 9,
                         1 : 9,
                         -1 : 0,
                         -2 : 0,
                        },
           "PAY_NOV" : {8 : 9,
                         7 : 9,
                         6 : 9,
                         5 : 9,
                         4 : 9,
                         3 : 9,
                         2 : 9,
                         1 : 9,
                         -1 : 0,
                         -2 : 0,
                        },
           "PAY_DEC" : {8 : 9,
                         7 : 9,
                         6 : 9,
                         5 : 9,
                         4 : 9,
                         3 : 9,
                         2 : 9,
                         1 : 9,
                         -1 : 0,
                         -2 : 0,
                        }
          }

In [ ]:
train["Balance_Limit_V1"] = train["Balance_Limit_V1"].astype('category')
train.replace(cleanup, inplace=True)
train["Balance_Limit_V1"] = train["Balance_Limit_V1"].cat.codes
train.head(100)

> # Model

In [ ]:
# import the libraries to train the model
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report

In [ ]:
y=train.NEXT_MONTH_DEFAULT

In [ ]:
train.drop(['Client_ID'], axis=1, inplace=True)
train.drop(['NEXT_MONTH_DEFAULT'], axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
#split the training data into train and cross validation set
X_train, X_valid, y_train, y_valid = train_test_split(train, y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
scaler.fit(train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
X_train

## Neural Network

In [ ]:
import tensorflow as tf
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.regularizers import l2

In [ ]:
model=Sequential()

In [ ]:
model.add(Dense(1024, activation = 'relu', input_dim = 23))
model.add(Dropout(0.2, noise_shape = None, seed = None))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2, noise_shape = None, seed = None))

model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
#train the model
model_output = model.fit(X_train, y_train,
                         epochs = 20, batch_size = 128,
                         verbose = 1,
                         validation_data = (X_valid, y_valid))

In [ ]:
_, accuracy = model.evaluate(X_train, y_train, batch_size=32)
_, accuracy_val = model.evaluate(X_valid, y_valid, batch_size=32)

In [ ]:
print('Accuracy: %.2f' % (accuracy*100)) #obtain the validation accuracy

In [ ]:
print('Accuracy of the validation set: %.2f' % (accuracy_val*100))

In [ ]:
yhat_classes = model.predict_classes(X_valid)
matrix = confusion_matrix(y_valid, yhat_classes)
matrix_nom=matrix.astype('float')/matrix.sum(axis=1)[:,np.newaxis]
print(matrix_nom)

In [ ]:
print(classification_report(yhat_classes, y_valid))#obtain the F1 score

In [ ]:
test.drop(['Client_ID'], axis=1, inplace=True)

In [ ]:
# cleanup the test set to make predictions
test["Balance_Limit_V1"] = test["Balance_Limit_V1"].astype('category')
test.replace(cleanup, inplace=True)
test["Balance_Limit_V1"] = test["Balance_Limit_V1"].cat.codes
test.head(100)

In [ ]:
scaler.fit(test)
test_=scaler.transform(test)
test_.shape

In [ ]:
predictions = model.predict_classes(test_)

In [ ]:
predictions=predictions.flatten()

# Generate submission

In [ ]:
submission = pd.DataFrame({'Client_ID':testc['Client_ID'],'NEXT_MONTH_DEFAULT':predictions})
submission.head()

In [ ]:
filename = 'NN.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)